<h4 style="color:#ff6600 ; text-align:left ; font-family: Verdana, serif;"><b>CNN with 12 layers (with Batch-Norm)</b></h4>

In [6]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import numpy
import json
import os

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

# custom import
from pyscripts.predict_class import test_mod

Using Theano backend.
WARNING (theano.configdefaults): Only clang++ is supported. With g++, we end up with strange g++/OSX bugs.


In [22]:
fl = './work/CNN_12/history.json'

with open(fl) as json_data:
    history = json.load(json_data)
trace0 = go.Scatter(
    x = range(len(history['acc'])),
    y = history['acc'],
    mode = 'lines+markers',
    name = 'Training Accuracy'    
)

trace1 = go.Scatter(
    x = range(len(history['val_acc'])),
    y = history['val_acc'],
    mode = 'lines+markers',
    name = 'Validation Accuracy'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Train-Validation Accuracy',
    xaxis=dict(
        title='Number of passes through data',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Accuracy',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!
py.iplot(fig, filename='traintest_acc')


In [23]:
trace0 = go.Scatter(
    x = range(len(history['loss'])),
    y = history['loss'],
    mode = 'lines+markers',
    name = 'Training Loss'    
)

trace1 = go.Scatter(
    x = range(len(history['val_loss'])),
    y = history['val_loss'],
    mode = 'lines+markers',
    name = 'Validation Loss'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Train-Validation Loss',
    xaxis=dict(
        title='Number of passes through data',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Loss',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!
py.iplot(fig, filename='traintest_loss')


<p style="color:black ; text-align:left ; font-family: Verdana, serif;"><b>Confusion Matrix for the Test data</b></p>

In [ ]:
path = './work'
model_name = 'CNN_12'

out_df, class_dictionary = test_mod(model_name, path)
out_df.to_csv(path + '/' + model_name + '/pred.csv', index=False)

In [27]:
df_in = pd.read_csv('./work/CNN_12/pred.csv')
cf = confusion_matrix(df_in['actual'], df_in['pred'])
x = class_dictionary.values()
y = class_dictionary.values()

In [29]:
''' Format of the confusion matrix / Heatmap
        x0    x1    x2    x3    x4
z = [[0.00, 0.00, 0.75, 0.75, 0.00],  y0
     [0.00, 0.00, 0.75, 0.75, 0.00],  y1
     [0.75, 0.75, 0.75, 0.75, 0.75],  y2
     [0.00, 0.00, 0.00, 0.75, 0.00]]  y3
'''

z = cf
annotations = []
for n, row in enumerate(z):
    for m, val in enumerate(row):
        var = z[n][m]
        annotations.append(
            dict(
                text=str(val),
                x=x[m], y=y[n],
                xref='x1', yref='y1',
                font=dict(color='white' if val > 0.5 else 'black'),
                showarrow=False)
            )

#colorscale = [[0, '#3D9970'], [1, '#001f3f']]  # custom colorscale
trace = go.Heatmap(x=x, y=y, z=z, colorscale='RdYlBu', showscale=True)

fig = go.Figure(data=[trace])
fig['layout'].update(
    title="Confusion matrix",
    annotations=annotations,
    xaxis=dict(title='Predicted',ticks='', side='down', tickangle=-45),
    # ticksuffix is a workaround to add a bit of padding
    yaxis=dict(title='Actual',ticks='', ticksuffix='  '),
    width=700,
    height=700,
    autosize=False,
    margin=go.Margin(
                    l=200,
                    b= 150,
                    pad=4
                    )
    
)
py.iplot(fig, filename='Confusion matrix', height=1000)

<p style="color:black ; text-align:left ; font-family: Verdana, serif;"><b>Classification Report for the Test data</b></p>

In [13]:
y_test = df_in['actual']
y_pred = df_in['pred']
cr = (classification_report(y_test, y_pred, target_names=class_dictionary.values()))
print cr

                           precision    recall  f1-score   support

   Cephalophus_harveyi_TZ       0.96      0.98      0.97       382
     Cercocebus_sanjei_TZ       0.95      1.00      0.98       122
   Dasyprocta_leporina_SR       0.93      0.99      0.96       155
  Nesotragus_moschatus_TZ       0.97      0.95      0.96       127
       Panthera_pardus_TZ       1.00      0.71      0.83        21
         Pecari_tajacu_SR       0.96      0.95      0.95       116
Potamochoerus_larvatus_TZ       0.98      0.95      0.97       163
     Psophia_crepitans_SR       0.99      0.93      0.96       162
        Tayassu_pecari_SR       0.96      0.98      0.97       130
  Tragelaphus_scriptus_TZ       0.96      0.90      0.93        30

              avg / total       0.96      0.96      0.96      1408



<p style="color:black ; text-align:left ; font-family: Verdana, serif;"><b>Overall Accuracy for the Test data</b></p>

In [14]:
accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)

0.96306818181818177

<h4 style="color:#ff6600 ; text-align:left ; font-family: Verdana, serif;"><b>CNN with 12 layers (with Batch-Norm + Dropout)</b></h4>

In [30]:
fl = './work/CNN_12_DO/history.json'

with open(fl) as json_data:
    history = json.load(json_data)
trace0 = go.Scatter(
    x = range(len(history['acc'])),
    y = history['acc'],
    mode = 'lines+markers',
    name = 'Training Accuracy'    
)

trace1 = go.Scatter(
    x = range(len(history['val_acc'])),
    y = history['val_acc'],
    mode = 'lines+markers',
    name = 'Validation Accuracy'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Train-Validation Accuracy',
    xaxis=dict(
        title='Number of passes through data',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Accuracy',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!
py.iplot(fig, filename='traintest_acc')


In [31]:
trace0 = go.Scatter(
    x = range(len(history['loss'])),
    y = history['loss'],
    mode = 'lines+markers',
    name = 'Training Loss'    
)

trace1 = go.Scatter(
    x = range(len(history['val_loss'])),
    y = history['val_loss'],
    mode = 'lines+markers',
    name = 'Validation Loss'
)

data = [trace0, trace1]
layout = go.Layout(
    title='Train-Validation Loss',
    xaxis=dict(
        title='Number of passes through data',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Loss',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!
py.iplot(fig, filename='traintest_loss')



<p style="color:black ; text-align:left ; font-family: Verdana, serif;"><b>Confusion Matrix for the Test data</b></p>

In [ ]:
path = './work'
model_name = 'CNN_12'

out_df, class_dictionary = test_mod(model_name, path)
out_df.to_csv(path + '/' + model_name + '/pred.csv', index=False)

In [32]:
df_in = pd.read_csv('./work/CNN_12_DO/pred.csv')
cf = confusion_matrix(df_in['actual'], df_in['pred'])
x = class_dictionary.values()
y = class_dictionary.values()

In [33]:
''' Format of the confusion matrix / Heatmap
        x0    x1    x2    x3    x4
z = [[0.00, 0.00, 0.75, 0.75, 0.00],  y0
     [0.00, 0.00, 0.75, 0.75, 0.00],  y1
     [0.75, 0.75, 0.75, 0.75, 0.75],  y2
     [0.00, 0.00, 0.00, 0.75, 0.00]]  y3
'''

z = cf
annotations = []
for n, row in enumerate(z):
    for m, val in enumerate(row):
        var = z[n][m]
        annotations.append(
            dict(
                text=str(val),
                x=x[m], y=y[n],
                xref='x1', yref='y1',
                font=dict(color='white' if val > 0.5 else 'black'),
                showarrow=False)
            )

#colorscale = [[0, '#3D9970'], [1, '#001f3f']]  # custom colorscale
trace = go.Heatmap(x=x, y=y, z=z, colorscale='RdYlBu', showscale=True)

fig = go.Figure(data=[trace])
fig['layout'].update(
    title="Confusion matrix",
    annotations=annotations,
    xaxis=dict(title='Predicted',ticks='', side='down', tickangle=-45),
    # ticksuffix is a workaround to add a bit of padding
    yaxis=dict(title='Actual',ticks='', ticksuffix='  '),
    width=700,
    height=700,
    autosize=False,
    margin=go.Margin(
                    l=200,
                    b= 150,
                    pad=4
                    )
    
)
py.iplot(fig, filename='Confusion matrix', height=1000)

<p style="color:black ; text-align:left ; font-family: Verdana, serif;"><b>Classification Report for the Test data</b></p>

In [18]:
y_test = df_in['actual']
y_pred = df_in['pred']
cr = (classification_report(y_test, y_pred, target_names=class_dictionary.values()))
print cr

                           precision    recall  f1-score   support

   Cephalophus_harveyi_TZ       0.79      0.75      0.77       382
     Cercocebus_sanjei_TZ       0.78      0.30      0.43       122
   Dasyprocta_leporina_SR       0.74      0.56      0.64       155
  Nesotragus_moschatus_TZ       0.81      0.70      0.75       127
       Panthera_pardus_TZ       0.27      0.14      0.19        21
         Pecari_tajacu_SR       0.29      0.93      0.44       116
Potamochoerus_larvatus_TZ       0.56      0.80      0.66       163
     Psophia_crepitans_SR       1.00      0.19      0.32       162
        Tayassu_pecari_SR       0.60      0.56      0.58       130
  Tragelaphus_scriptus_TZ       1.00      0.03      0.06        30

              avg / total       0.72      0.60      0.59      1408



<p style="color:black ; text-align:left ; font-family: Verdana, serif;"><b>Overall Accuracy for the Test data</b></p>

In [20]:
accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)

0.60014204545454541